<a href="https://colab.research.google.com/github/mdsaifin/vita/blob/main/VITA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# VITA
# iBuild HAckathon
# By MD Saif and Aditya

In [2]:
# 1) Voice to text: Whisper from OpenAI
# 2) Text to Voice: gTTS
# 3) Intent Analysis: To bucket all the questions based on their intent
# 4) Keyboard Matching: From that Intent, finding question with highest keyword match
# 5) Providing answer in audio mode from that question

# Apart of MVP, we will add:
# 1) Voice Synthesis for better user experience and low bounce rate: ElevenLabs and Bark AI
# 2) Using GenAI for providing answers on the same format as user asks: Text Generation (LLMs) Using Llama 2
# 3) Call Memormemorization: Concatination of new keywords of next questions in previous questions to store memory and provide relevant answers
# 4) Multi-lang Integration : langDetect(language detection) + WhisperAI + HF MarianMT + gTTS / BarkAI
# 5) Emotional Integration : OpenSource - Speech Emotion : HF Wav2Vec2 + EmoNet
                                       #  Text Enmotion : HF GoEmotions + DistilBERT

In [ ]:
#installing libraries

In [ ]:
!pip install -q openai-whisper torch ffmpeg numpy pydub
!pip install -q spacy transformers gtts webrtcvad
!python -m spacy download en_core_web_sm
!apt update && apt install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install mutagen
import whisper
import IPython
from google.colab import output
from base64 import b64decode
from gtts import gTTS
from IPython.display import Audio,display
import pandas as pd
import pandas as pd

import time
import numpy as np
import scipy.io.wavfile as wav
import webrtcvad
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import IPython
from IPython.display import Javascript
from base64 import b64decode
from google.colab import output

!pip install -q openai-whisper torch ffmpeg numpy pydub

# Install ffmpeg (for audio processing)
!apt update && apt install -y ffmpeg

!pip install spacy
!pip install transformers
!python -m spacy download en_core_web_sm
import spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.2 MB/s eta 0:00:00
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
24 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'ma

In [ ]:
model = whisper.load_model("base")

# Load the zero-shot classification pipeline for intent detection - bucket is forming for intent detection
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Define candidate labels for customer intent - defining bucket
candidate_labels = [
    "Greeting_Permission",
    "Patient_Symptom_Profiling",
    "Program_Explanation",
    "Relief_Inquiry",
    "Objection_Handling",
    "Closing_Conversion",
    "Skepticism",
    "Cost",
    "Commitment"
]

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 166MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Preprocess text for uniformity (e.g., lowercase) for better accuracy
def preprocess_text(text):
    return text.lower()

# Function to classify intent using zero-shot classification
def classify_intent(text):
    result = classifier(text, candidate_labels)
    return result['labels'][0], result['scores'][0]  # most likely intent and score

# Function to compute cosine similarity and find best matching predefined question
def find_best_match(input_question, intent_questions):
    input_question = preprocess_text(input_question)
    best_match = None
    best_match_score = 0
    matched_intent = None

    # Loop through each intent and its associated questions (keys of the inner dict)
    for intent, questions in intent_questions.items():
        for question in questions:
            example_question = preprocess_text(question)
            vectorizer = CountVectorizer().fit_transform([input_question, example_question])
            cosine_sim = cosine_similarity(vectorizer[0:1], vectorizer[1:2])
            if cosine_sim[0][0] > best_match_score:
                best_match_score = cosine_sim[0][0]
                best_match = question
                matched_intent = intent
    return matched_intent, best_match, best_match_score

# Function for converting text to speech using gTTS
def text_to_speech(text, language="en"):
    speech = gTTS(text=text, lang=language, slow=False)
    return speech

# Function to play the generated speech audio
def play_audio(speech, filename="agent_answer.mp3"):
    speech.save(filename)
    return Audio(filename, autoplay=True)

In [ ]:
#New voice

!pip install edge-tts
import asyncio
import edge_tts
from IPython.display import Audio
async def text_to_speech2(text, voice="en-US-JennyNeural", output_file="output_audio.mp3"):
    tts = edge_tts.Communicate(text, voice)
    await tts.save(output_file)
    return output_file
def play_audio2(file):
    return Audio(file, autoplay=True)
import nest_asyncio
nest_asyncio.apply()

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=e5258532aa429790c98f148a59257bc28b3cfbfe2003215b04c507e57449cdb5
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [ ]:
async def run_tts(text):
    output_file = await text_to_speech2(text)
    display(play_audio2(output_file))

In [ ]:
#User name

!pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet using the URL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1008Jmjn6MkH4CfGZqZFsaS7j2qfO8jp6uQfINnf4xnE/edit?usp=sharing"
spreadsheet = gc.open_by_url(spreadsheet_url)

# Select the first worksheet
worksheet = spreadsheet.get_worksheet(0)  # Assuming First_Name is in the first sheet

# Fetch the data from the second column (Column B)
first_names = worksheet.col_values(2)  # Column index 2 corresponds to First_Name

# Store the first name (assuming you need the first row after the header)
user_name = first_names[1]  # Skipping the header, assuming first row has the name

In [ ]:
# Print the fetched first name
print("User Name:", user_name)

User Name: Saif


In [ ]:
# based on our research according to the script (documentation) -- can be added more to for better accuracy
intent_answers = {

  "greeting_permission": {
    "hello? who is this?": f"hi {user_name}, this is vita, your ai health assistant from dailybloom. i’m reaching out with a quick health check-in. do you have a minute to discuss your gut health?",
    "hi? who is this?": f"Hi {user_name}, this is Vita, from DailyBloom IBS by Dr. Reddy. I am reaching out because you recently showed interest in IBS solutions. Is this a good time to talk?",
    "yes, sure. what is this about?": "thank you! i’d like to check if you’ve been experiencing any gut-related symptoms. may i ask if you frequently experience bloating, constipation, diarrhea, or stomach discomfort?"
  },
  "patient_symptom_profiling": {
    "yes, i do get bloating sometimes.": "i understand. bloating can sometimes indicate an imbalance in gut health. do you also notice any irregular bowel movements, such as frequent diarrhea or constipation?",
    "yes, i have constipation quite often.": "noted. chronic constipation can sometimes be linked to ibs or other digestive concerns. have you also noticed any food intolerances, fatigue, or stress-related gut issues?",
    "yes, i feel my gut reacts when i’m stressed.": "that’s insightful. stress can impact gut health significantly. based on what you’ve shared, it might be beneficial to explore solutions for better digestive health."
  },
  "program_explanation": {
    "what is the duration of your program " : "We have a 14-week program which provides IBS expert nutritionists and wellness coaches - who will help identify and overcome your food and stress triggers . We also offer Yoga modules to reduce your  symptoms  This program is split into 4 phases: Discovery, Gut cleanse, Reintroduction and  Sustenance ",
    "what kind of solution you provide ?": "dailybloom offers a clinically researched ibs care program designed to help manage symptoms naturally. it includes personalized nutrition plans, probiotic support, and expert guidance. would you like to hear how it works?",
    "yes, tell me more.": "great! our program is designed by gut health specialists. it focuses on improving digestion through diet adjustments, probiotics, and stress management techniques. many of our users have seen significant relief.",
    "is this a medicine?": "no, dailybloom is not a medication. it’s a holistic program that helps improve gut health naturally through lifestyle and dietary changes, combined with expert support.",
    "i have tried many things before. how is this different?": "that’s a valid concern! unlike generic solutions, dailybloom offers a customized approach tailored to your specific symptoms and lifestyle, ensuring better and lasting results.",
    "that sounds interesting. how can i start?": "i’m glad you’re interested! we offer a free initial consultation with a gut health expert. would you like me to schedule a quick 10-minute call at your convenience?",
    "yes, that would be good.": "perfect! i’ll book your free consultation. you’ll receive a confirmation via whatsapp shortly. looking forward to helping you on your gut health journey!"
  },
  "relief_inquiry": {
    "will this actually help with my bloating and constipation?": "yes! many of our users have experienced significant relief from bloating and constipation through our personalized gut health program. our approach focuses on identifying triggers and making targeted dietary and lifestyle changes.",
    "how soon can i expect results?": "results vary from person to person, but most users start noticing improvements within a few weeks. our expert guidance ensures a structured plan tailored to your symptoms for better long-term relief.",
    "do you have any success stories?": "absolutely! many of our users have successfully managed their ibs symptoms with our program. we have testimonials from people who have reduced bloating, improved digestion, and gained more energy through our expert-led approach.",
    "is this program backed by doctors?": "yes! our program is developed by gut health specialists and nutrition experts who have helped many individuals manage ibs symptoms effectively."
  },
  "objection_handling": {
    "i’m not sure if i can commit to a full program.": "i understand! that’s why we offer a flexible approach. you can start with small, manageable steps, and our team will support you along the way to make it easy and sustainable.",
    "i’ve tried other programs before, and they didn’t work.": "that makes sense, and it’s completely valid to feel that way. what sets dailybloom apart is our personalized approach—we tailor the plan specifically to your gut health needs rather than offering a one-size-fits-all solution.",
    "is this expensive?": "our program is designed to be affordable while offering premium guidance. plus, we offer a free initial consultation so you can understand the value before making any commitment.",
    "i don't have time for this.": "i get that! that’s why our program is designed to fit into your routine without major disruptions. we offer simple, actionable steps that don’t require a lot of time but can still improve your gut health.",
    "i’m not sure if my symptoms are severe enough.": "that’s a good point! even mild symptoms can be a sign of gut imbalances. identifying them early can help prevent more serious digestive issues later. would you like to explore a free consultation to understand your gut health better?"
  },
  "closing_conversion": {
    "this sounds good, but i need to think about it.": "i completely understand! to help you decide, we can set up a free consultation where our experts can address your concerns and guide you through the process. would you like me to schedule one for you?",
    "okay, let’s do the consultation.": "great! i’ll book your free consultation with our gut health expert. you’ll receive a confirmation via whatsapp shortly. looking forward to helping you improve your gut health!",
    "thanks, i’ll check it out.": "you’re welcome! i’ll send you all the details via whatsapp. if you have any questions, feel free to reach out. wishing you the best on your gut health journey!",
    "what happens after the consultation?": "after the consultation, our expert will provide personalized insights about your gut health. if you’re interested, you can then choose to join the program with a tailored plan designed just for you."
  },
  "skepticism": {
    "has this worked for other people like me": "absolutely! we’ve helped thousands of people with different ibs symptoms, and many see up to 100% improvement in 4 weeks. each plan is personalized based on your symptoms and triggers.",
    "that sounds complicated.": "i completely understand. many of our members felt the same way, but they saw major improvements by addressing root causes with expert guidance.",
    "how do i know this will work for me?": "that’s a great question. every person’s ibs is unique, which is why we personalize the program based on your specific triggers. thousands of people with different symptoms have followed our program and experienced lasting relief.",
    "i don’t think diet can help my ibs.": "actually, food and stress are two major ibs triggers. our program combines personalized nutrition with stress management for better, lasting relief.",
    "nothing ever helps, why would this be different?": "this isn’t a one-size-fits-all program—we tailor it to your unique triggers and symptoms. plus, you’ll have expert guidance every step of the way.",
    "other programs also claim results, how is this one better?": "unlike generic solutions, we focus on identifying your personal triggers, reintroducing foods safely, and providing expert support throughout the process."
  },
  "cost": {
    "What’s the price of your plan?" : "sure ,to keep it simple, you can start with the first two phases for just ₹2999. In these four weeks, you'll see real improvements in your gut health while experiencing the expertise of our coaches. If you love the results, you can continue the full program for around ₹5000. We’re confident you’ll benefit from it—that’s why we’re offering this special trial! 😊",
    "why is the first phase more expensive?": "great question. the first four weeks require more expert support to reset your gut, which is why we work closely with you during this period. the goal is to give you significant relief in this critical phase so you can decide if you want to continue.",
    "i’ve already spent so much on treatments.": "i understand how frustrating it is to invest in treatments that don’t work. that’s why we designed this program to be both affordable and effective, so you don’t have to waste money on trial-and-error solutions."
  },
  "commitment": {
    "i've tried everything, nothing works.": "i totally get that frustration! what’s different about our approach is that we don’t just offer temporary fixes—we help you build lasting solutions step by step.",
    "i don’t know what’s causing my ibs. could be stress, could be food.": "you’re not alone in feeling that way! ibs is usually linked to food triggers, stress, and gut health imbalances. our program helps pinpoint the cause and gradually reintroduces foods that work for your body.",
    "my doctor told me i have ibs, but nothing has worked.": "i completely understand. many of our members felt the same way, but they saw major improvements by addressing root causes with expert guidance.",
    "i've tried probiotics and medications, but i still have symptoms.": "medications can help manage symptoms, but our approach focuses on long-term relief by restoring balance and improving gut resilience.",
    "i have ibs for years; it’s not going to go away now.": "ibs is a long-term condition, but that doesn’t mean you have to suffer. many people with chronic ibs have dramatically reduced their symptoms with our approach.",
    "i've taken expert advice before, and i'm still struggling.": "that’s completely understandable. ibs is complex, and no single approach works for everyone. that’s why we customize our program to fit your specific needs.",
    "i just want a quick fix, not a long-term program.": "i totally get it! that’s why we focus on noticeable improvements from week 1, with many members seeing significant results in just four weeks."
  }

    }

In [ ]:
def request_microphone_access():
    REQUEST_MIC_JS = """
    navigator.mediaDevices.getUserMedia({ audio: true }).then(() => {
        alert("Microphone access granted! Now you can start recording.");
    }).catch(error => {
        alert("Microphone access denied! Please allow it in your browser settings and refresh.");
    });
    """
    IPython.display.display(IPython.display.Javascript(REQUEST_MIC_JS))

def record_audio():
    RECORD_AUDIO_JS = """
    const sleep = time => new Promise(resolve => setTimeout(resolve, time));

    async function recordAudio() {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            const mediaRecorder = new MediaRecorder(stream);
            const audioChunks = [];

            let silenceTimeout;
            const silenceThreshold = 15;
            const silenceDuration = 1000;

            const audioContext = new AudioContext();
            const analyser = audioContext.createAnalyser();
            const source = audioContext.createMediaStreamSource(stream);
            source.connect(analyser);
            analyser.fftSize = 512;

            mediaRecorder.ondataavailable = event => {
                audioChunks.push(event.data);
            };

            function checkSilence() {
                const bufferLength = analyser.frequencyBinCount;
                const dataArray = new Uint8Array(bufferLength);
                analyser.getByteFrequencyData(dataArray);
                const volume = dataArray.reduce((a, b) => a + b) / bufferLength;
                console.log("Volume Level:", volume);

                if (volume < silenceThreshold) {
                    if (!silenceTimeout) {
                        console.log("Silence detected... stopping soon");
                        silenceTimeout = setTimeout(() => {
                            console.log("Stopping recording due to silence");
                            mediaRecorder.stop();
                        }, silenceDuration);
                    }
                } else {
                    clearTimeout(silenceTimeout);
                    silenceTimeout = null;
                }
            }

            mediaRecorder.start();
            console.log("Recording... Speak now!");

            const interval = setInterval(checkSilence, 500);

            mediaRecorder.onstop = () => {
                clearInterval(interval);
                console.log("Recording stopped!");
            };

            await new Promise(resolve => mediaRecorder.onstop = resolve);

            const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
            const reader = new FileReader();
            reader.readAsDataURL(audioBlob);
            reader.onloadend = () => {
                const base64data = reader.result.split(',')[1];
                google.colab.kernel.invokeFunction('notebook.record_audio', [base64data], {});
            };
        } catch (error) {
            console.error("Error accessing microphone:", error);
            alert("Microphone access failed. Please check your permissions.");
        }
    }

    console.log("Click to start recording");
    recordAudio();
    """
    print("Speak now. The system will automatically stop recording after detecting silence.")
    IPython.display.display(IPython.display.Javascript(RECORD_AUDIO_JS))

def save_audio(b64data):
    audio_path = "/content/recorded_audio.wav"
    with open(audio_path, "wb") as f:
        f.write(b64decode(b64data))
    print("▶️Audio recorded and saved as:", audio_path)
    print("🎗️🎗️You can now transcribe it using Whisper!")

# Corrected: Use google.colab.output.register_callback
output.register_callback('notebook.record_audio', save_audio)

# Usage Example
# request_microphone_access()  # Call once to request access
# record_audio()  # Call multiple times to start recording

request_microphone_access()

<IPython.core.display.Javascript object>

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file = "/content/recorded_audio.wav"
result = model.transcribe(audio_file)
print("\n📝 Transcription:", result["text"])

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Hello with this


In [ ]:
response = result["text"]

In [ ]:
def extract_transcribed_text(text, result):
    transcribed_text = result["text"]
    input_question = transcribed_text
    responses = [input_question]

    return input_question, responses

In [ ]:
extract_transcribed_text("text", result)

(' Hello with this', [' Hello with this'])

In [ ]:
print(response)

 Hello with this


In [ ]:
nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    doc = nlp(text)
    preprocessed_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    return preprocessed_text

In [ ]:
def preprocess_and_print_responses(responses, preprocess_text):
    preprocessed_responses = []

    print("\nPreprocessing Responses:")
    for response in responses:
        preprocessed = preprocess_text(response)
        preprocessed_responses.append(preprocessed)
        print(f"Original: {response}")
        print(f"Preprocessed: {preprocessed}")
        print()

    return preprocessed_responses

In [ ]:
def process_customer_responses(responses, input_question, intent_answers):
    print("Classifying Intent for each Response:")
    classified_intents = []

    for response in responses:
        intent, score = classify_intent(response)
        classified_intents.append((response, intent, score))
        print(f"Customer Response: {response}")
        print(f"Predicted Intent: {intent} with score: {score}")
        print()

    # Find the best matching question (and thus intent) from our predefined answers
    matched_intent, best_match, match_score = find_best_match(input_question, intent_answers)
    answer = intent_answers.get(matched_intent, {}).get(best_match, "Sorry, I don't have an answer for that.")

    # Display the matching details and answer
    print("\n--- Intent Matching Details ---")
    print(f"Input Question: {input_question}")
    print(f"Matched Intent: {matched_intent}")
    print(f"Best Match Question: {best_match}")
    print(f"Cosine Similarity Score: {match_score}")
    print(f"Answer: {answer}")

    return matched_intent, best_match, match_score, answer, classified_intents

In [ ]:
def get_best_match_answer(input_question, intent_answers):
    matched_intent, best_match, match_score = find_best_match(input_question, intent_answers)
    answer = intent_answers.get(matched_intent, {}).get(best_match, "Sorry, I don't have an answer for that.")

    return answer

In [ ]:
preprocess_and_print_responses(response, preprocess_text)


Preprocessing Responses:
Original:  
Preprocessed:  

Original: H
Preprocessed: h

Original: e
Preprocessed: e

Original: l
Preprocessed: l

Original: l
Preprocessed: l

Original: o
Preprocessed: o

Original:  
Preprocessed:  

Original: w
Preprocessed: w

Original: i
Preprocessed: 

Original: t
Preprocessed: t

Original: h
Preprocessed: h

Original:  
Preprocessed:  

Original: t
Preprocessed: t

Original: h
Preprocessed: h

Original: i
Preprocessed: 

Original: s
Preprocessed: s



[' ', 'h', 'e', 'l', 'l', 'o', ' ', 'w', '', 't', 'h', ' ', 't', 'h', '', 's']

In [ ]:
process_customer_responses(response, input_question, intent_answers)

Classifying Intent for each Response:
Customer Response:  
Predicted Intent: Commitment with score: 0.15891537070274353

Customer Response: H
Predicted Intent: Commitment with score: 0.18731766939163208

Customer Response: e
Predicted Intent: Commitment with score: 0.1557946503162384

Customer Response: l
Predicted Intent: Commitment with score: 0.1539672613143921

Customer Response: l
Predicted Intent: Commitment with score: 0.1539672613143921

Customer Response: o
Predicted Intent: Objection_Handling with score: 0.13221225142478943

Customer Response:  
Predicted Intent: Commitment with score: 0.15891537070274353

Customer Response: w
Predicted Intent: Commitment with score: 0.1661110371351242

Customer Response: i
Predicted Intent: Commitment with score: 0.17106924951076508

Customer Response: t
Predicted Intent: Commitment with score: 0.17114359140396118

Customer Response: h
Predicted Intent: Commitment with score: 0.1493259221315384

Customer Response:  
Predicted Intent: Commitm

('cost',
 'why is the first phase more expensive?',
 0.9999999999999998,
 'great question. the first four weeks require more expert support to reset your gut, which is why we work closely with you during this period. the goal is to give you significant relief in this critical phase so you can decide if you want to continue.',
 [(' ', 'Commitment', 0.15891537070274353),
  ('H', 'Commitment', 0.18731766939163208),
  ('e', 'Commitment', 0.1557946503162384),
  ('l', 'Commitment', 0.1539672613143921),
  ('l', 'Commitment', 0.1539672613143921),
  ('o', 'Objection_Handling', 0.13221225142478943),
  (' ', 'Commitment', 0.15891537070274353),
  ('w', 'Commitment', 0.1661110371351242),
  ('i', 'Commitment', 0.17106924951076508),
  ('t', 'Commitment', 0.17114359140396118),
  ('h', 'Commitment', 0.1493259221315384),
  (' ', 'Commitment', 0.15891537070274353),
  ('t', 'Commitment', 0.17114359140396118),
  ('h', 'Commitment', 0.1493259221315384),
  ('i', 'Commitment', 0.17106924951076508),
  ('s', 'C

**Story Building**

In [ ]:
#call connected

USER SPEECH-1

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U1 = "/content/recorded_audio_U1.wav"
result_U1 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U1["text"])
text_U1 = result_U1["text"]

response_U1 = result_U1["text"]
extract_transcribed_text(text, result_U1)
transcribed_text = preprocess_text(text_U1)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Hello who is this?


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U1, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   hello ?
Preprocessed:    hello ?

Classifying Intent for each Response:
Customer Response:   hello ?
Predicted Intent: Relief_Inquiry with score: 0.3439464867115021


--- Intent Matching Details ---
Input Question:   hello ?
Matched Intent: greeting_permission
Best Match Question: hello? who is this?
Cosine Similarity Score: 1.0
Answer: hi Saif, this is vita, your ai health assistant from dailybloom. i’m reaching out with a quick health check-in. do you have a minute to discuss your gut health?


USER SPEECH-2

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U2 = "/content/recorded_audio_U2.wav"
result_U2 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U2["text"])
text_U2 = result_U2["text"]

response_U2 = result_U2["text"]
extract_transcribed_text(text, result_U2)
transcribed_text = preprocess_text(text_U2)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Yes, I checked your website. Can you tell me more about this?


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U2, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   yes , check website . tell ?
Preprocessed:    yes , check website . tell ?

Classifying Intent for each Response:
Customer Response:   yes , check website . tell ?
Predicted Intent: Relief_Inquiry with score: 0.3827495574951172


--- Intent Matching Details ---
Input Question:   yes , check website . tell ?
Matched Intent: program_explanation
Best Match Question: yes, tell me more.
Cosine Similarity Score: 0.7071067811865475
Answer: great! our program is designed by gut health specialists. it focuses on improving digestion through diet adjustments, probiotics, and stress management techniques. many of our users have seen significant relief.


USER SPEECH-3

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U3 = "/content/recorded_audio_U3.wav"
result_U3 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U3["text"])
text_U3 = result_U3["text"]

response_U3 = result_U3["text"]
extract_transcribed_text(text, result_U3)
transcribed_text = preprocess_text(text_U3)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Yes, I also suffer from constipation.


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U3, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   yes , suffer constipation .
Preprocessed:    yes , suffer constipation .

Classifying Intent for each Response:
Customer Response:   yes , suffer constipation .
Predicted Intent: Relief_Inquiry with score: 0.3309984803199768


--- Intent Matching Details ---
Input Question:   yes , suffer constipation .
Matched Intent: patient_symptom_profiling
Best Match Question: yes, i have constipation quite often.
Cosine Similarity Score: 0.816496580927726
Answer: noted. chronic constipation can sometimes be linked to ibs or other digestive concerns. have you also noticed any food intolerances, fatigue, or stress-related gut issues?


USER SPEECH-4

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U4 = "/content/recorded_audio_U4.wav"
result_U4 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U4["text"])
text_U4 = result_U4["text"]

response_U4 = result_U4["text"]
extract_transcribed_text(text, result_U4)
transcribed_text = preprocess_text(text_U4)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Yes, my gut reacts when I get stressed.


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U4, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   yes , gut react stress .
Preprocessed:    yes , gut react stress .

Classifying Intent for each Response:
Customer Response:   yes , gut react stress .
Predicted Intent: Patient_Symptom_Profiling with score: 0.24019765853881836


--- Intent Matching Details ---
Input Question:   yes , gut react stress .
Matched Intent: patient_symptom_profiling
Best Match Question: yes, i feel my gut reacts when i’m stressed.
Cosine Similarity Score: 0.8944271909999159
Answer: that’s insightful. stress can impact gut health significantly. based on what you’ve shared, it might be beneficial to explore solutions for better digestive health.


USER SPEECH-5

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U5 = "/content/recorded_audio_U5.wav"
result_U5 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U5["text"])
text_U5 = result_U5["text"]

response_U5 = result_U5["text"]
extract_transcribed_text(text, result_U5)
transcribed_text = preprocess_text(text_U5)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Is there any kind of medicine?


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U5, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   kind medicine ?
Preprocessed:    kind medicine ?

Classifying Intent for each Response:
Customer Response:   kind medicine ?
Predicted Intent: Relief_Inquiry with score: 0.3167615532875061


--- Intent Matching Details ---
Input Question:   kind medicine ?
Matched Intent: program_explanation
Best Match Question: is this a medicine?
Cosine Similarity Score: 0.7071067811865475
Answer: no, dailybloom is not a medication. it’s a holistic program that helps improve gut health naturally through lifestyle and dietary changes, combined with expert support.


USER SPEECH-6

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U6 = "/content/recorded_audio_U6.wav"
result_U6 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U6["text"])
text_U6 = result_U6["text"]

response_U6 = result_U6["text"]
extract_transcribed_text(text, result_U6)
transcribed_text = preprocess_text(text_U6)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Is it expensive?


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U6, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   expensive ?
Preprocessed:    expensive ?

Classifying Intent for each Response:
Customer Response:   expensive ?
Predicted Intent: Cost with score: 0.6310489177703857


--- Intent Matching Details ---
Input Question:   expensive ?
Matched Intent: objection_handling
Best Match Question: is this expensive?
Cosine Similarity Score: 1.0
Answer: our program is designed to be affordable while offering premium guidance. plus, we offer a free initial consultation so you can understand the value before making any commitment.


USER SPEECH-7

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U7 = "/content/recorded_audio_U7.wav"
result_U7 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U7["text"])
text_U7 = result_U7["text"]

response_U7 = result_U7["text"]
extract_transcribed_text(text, result_U7)
transcribed_text = preprocess_text(text_U7)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  I have tried many things but nothing works.


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U7, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   try thing work .
Preprocessed:    try thing work .

Classifying Intent for each Response:
Customer Response:   try thing work .
Predicted Intent: Relief_Inquiry with score: 0.20836761593818665


--- Intent Matching Details ---
Input Question:   try thing work .
Matched Intent: commitment
Best Match Question: i've tried everything, nothing works.
Cosine Similarity Score: 0.816496580927726
Answer: i totally get that frustration! what’s different about our approach is that we don’t just offer temporary fixes—we help you build lasting solutions step by step.


USER SPEECH-8

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U8 = "/content/recorded_audio_U8.wav"
result_U8 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U8["text"])
text_U8 = result_U8["text"]

response_U8 = result_U8["text"]
extract_transcribed_text(text, result_U8)
transcribed_text = preprocess_text(text_U8)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Okay, thanks. I will check it out.


In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U8, intent_answers))

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Preprocessing Responses:
Original:   okay , thank . check .
Preprocessed:    okay , thank . check .

Classifying Intent for each Response:
Customer Response:   okay , thank . check .
Predicted Intent: Greeting_Permission with score: 0.28073692321777344


--- Intent Matching Details ---
Input Question:   okay , thank . check .
Matched Intent: closing_conversion
Best Match Question: thanks, i’ll check it out.
Cosine Similarity Score: 0.816496580927726
Answer: you’re welcome! i’ll send you all the details via whatsapp. if you have any questions, feel free to reach out. wishing you the best on your gut health journey!


In [ ]:
#Call disconnected from user side

Demo

In [ ]:
record_audio()

Speak now. The system will automatically stop recording after detecting silence.


<IPython.core.display.Javascript object>

▶️Audio recorded and saved as: /content/recorded_audio.wav
🎗️🎗️You can now transcribe it using Whisper!


In [ ]:
audio_file_U9 = "/content/recorded_audio_U9.wav"
result_U9 = model.transcribe(audio_file)
print("\n📝 Transcription:", result_U9["text"])
text_U9 = result_U9["text"]

response_U9 = result_U9["text"]
extract_transcribed_text(text, result_U9)
transcribed_text = preprocess_text(text_U9)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcription:  Why this program is too long?


NameError: name 'text' is not defined

In [ ]:
process_transcribed_text(transcribed_text, intent_answers)
await run_tts(get_best_match_answer(text_U9, intent_answers))